### Your First Flask Application

Let's create a simple webpage that will just show Hello, World!

We will build our app gradually: The first version will be static app.

In [8]:
%%writefile hello.py
from flask import Flask

app = Flask(__name__)


@app.route('/')
def hello():
    return 'Hello, World! This is my first flask app'

@app.route('/login')
def login():
    return 'No passwords, all welecome'

@app.route('/predict')
def predict():
    return 'No Snow Tomorrow!'

if __name__ == '__main__':
    app.run()


Overwriting hello.py


> app = `Flask(__name__)` will creates the Flask instance

>` __name__` is the name of the current Python module

> route is a `decorator` that connects the url to a function

> In pyhton, `decorator` is a function that modify the functionality of other functions 

> Note that we have several routes that are connected to different functins. For example, if the user accessed `/predict` route, the app will return `No Snow Tomorrow!`, you can think of theses different routes as the different links within the same webapage.


**Class discussion**:
- What do you see when you test different routes?
- What do you think the next level of complexity should be for this app to be more interesting.

### HTML Escaping

In [17]:
%%writefile hello.py
from flask import Flask
from markupsafe import escape


app = Flask(__name__)


@app.route('/')
def hello():
    return 'Hello, World! This is my first flask app'

@app.route('/login')
def login():
    return 'No passwords, all welecome'

@app.route('/user/<username>')
def show_user_profile(username):
    # show the user profile for that user
    return f'Welcome {escape(username)}'
if __name__ == '__main__':
    app.run()



Overwriting hello.py


### Rendering a template

We can use`render_template()` method to render html templates. We need provide the name of the template and the variables we want to pass to the template engine as keyword arguments. 

In [42]:
%%writefile hello.py


from flask import render_template
from flask import Flask
from markupsafe import escape


app = Flask(__name__)


@app.route('/')
def hello():
    return 'Hello, World! This is my first flask app'

@app.route('/login')
def login():
    return 'No passwords, all welecome'

@app.route('/welcome/')
@app.route('/welcome/<name>')
def welcome(name=None):
    return render_template('welcome.html', name=name)
if __name__ == '__main__':
    app.run()


Overwriting hello.py


### A trivial app

Now, let's think about the ML-powered app that we want to build. Essentially we need the user to interact with the app in 3 ways:

- Provide inputs
- Run the model
- Get the outputs

To interact with the app, we need two mainly types of methods

1. Post method
2. Get method



We will use the POST method requests to supply additional data from the client (browser) to the server in the message body.

In [46]:
%%writefile hello.py


from flask import render_template
from flask import Flask
from markupsafe import escape
from flask import request



app = Flask(__name__)


@app.route('/')
def hello():
    return 'Hello, World! This is my first flask app'

@app.route('/predict', methods=['GET', 'POST'])
def predict():
    if request.method == 'POST':
        thismonth = request.form['thismonth']
        thisyear = request.form['thisyear']
        return f"We are in {thismonth}, {thisyear}. Enjoy what is left of it."
    return '''
        <h1> My Trivial App </h1>
    
        <form method="post">
            <p><input type=text name=thismonth placeholder="Enter This Month Value" >
             <p><input type=text name=thisyear placeholder="Enter This Year Value">
            <p><input type=submit value=predict>
        </form>
    '''

@app.route('/welcome/')
@app.route('/welcome/<name>')
def welcome(name=None):
    return render_template('welcome.html', name=name)


if __name__ == '__main__':
    app.run()



Overwriting hello.py


### The full app

Let's take our app to the next level. I will be using a model that I developed before to predict the interfacial tension IFT between oil and water when CO2 or CH4 is added to the mixture. To read more about this work, you can visit the [project link](https://github.com/dataubc/Modeling_IFT_for_heavy_oil_emulsion)

for all we care about for model deployment, you just need to know that this model takes 4 inputs, namely:

- Gas type, CO2 or CH4
- Water content in the mixture, fraction between 0 and 1
- The viscosity of the oil used in the mixture

And will give back one output will be the the IFT

In [54]:
%%writefile hello.py


from flask import render_template
from flask import Flask
from markupsafe import escape
from flask import request
from joblib import load
import pandas as pd

app = Flask(__name__)

model_path = "models/model.pkl"
with open(model_path, 'rb') as file:
    model = load(file)

@app.route('/')
def hello():
    return 'Hello, World! This is my first flask app'

@app.route('/predict', methods=['GET', 'POST'])
def predict():
    if request.method == 'POST':
        gas_type = request.form['gas_type']
        water_content = float(request.form['water_content'])
        viscosity = float(request.form['viscosity'])
        time_minutes = float(request.form['time_minutes'])
        inputs = [[gas_type,water_content,viscosity,time_minutes]]
        df = pd.DataFrame(inputs, columns=['Gas', 'Water_content',
                                         'viscosity','time_minutes'])
        prediction = model.predict(df)[0]
        prediction = round(prediction, 2)
        
        return render_template('predict.html', prediction_text='Estimated IFT =  {} mN/m'.format( prediction))
    

@app.route('/welcome/')
@app.route('/welcome/<name>')
def welcome(name=None):
    return render_template('welcome.html', name=name)


if __name__ == '__main__':
    app.run()




Overwriting hello.py
